In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
# Define the network

class Net(nn.Module):
  
  def __init__(self):
    super(Net, self).__init__()
    
    self.conv1 = nn.Conv2d(1, 6, 3) # 1 input image channel, 6 output channels, 3x3 square convolution kernel
    self.conv2 = nn.Conv2d(6, 16, 3)
    
    self.fc1 = nn.Linear(16*6*6, 120) # 6*6 from image dimension
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
    
    
  def forward(self, x):
    # Max pooling over a (2,2) window
    x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    x = x.view(-1, self.num_flat_features(x)) # view function is used to reshape a tensor
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x
  
  def num_flat_features(self, x):
    size = x.size()[1:] # all dimensions except the batch dimension
    num_features = 1
    for s in size:
      num_features *= s
    return num_features    

In [16]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [0]:
# the backward function is automatically defined using autograd()

In [18]:
net.parameters() # retuns the learnable parameters

<generator object Module.parameters at 0x7f10b62ceca8>

In [19]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # weights of conv1

10
torch.Size([6, 1, 3, 3])


In [24]:
# the input size for the NN  is 32x32

input = torch.randn(1, 1, 32, 32) # torch.nn does not support single samples, but only minibatches of one or more samples
out = net(input)
print(out)

tensor([[-0.0230,  0.0344,  0.0374,  0.0749,  0.0750, -0.1744,  0.0148, -0.1050,
         -0.0839, -0.1194]], grad_fn=<AddmmBackward>)


In [0]:
# backpropagation with random gradients

out.backward(torch.randn(1, 10))

In [26]:
# computing loss

output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.5314, grad_fn=<MseLossBackward>)


In [31]:
print(loss.grad_fn) # MSE loss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

In [32]:
# Backprop

# loss.backward() will automatically implement backpropagation

# First we need to clear the existing gradients, else new gradients will accumuate to the existing gradients

net.zero_grad() # clears all the gradient buffers

print('conv1.bias.grad before backprop')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad afrer backprop')
print(net.conv1.bias.grad)

conv1.bias.grad before backprop
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad afrer backprop
tensor([-0.0050,  0.0081,  0.0047,  0.0084,  0.0078, -0.0180])


In [0]:
# weight update

import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

# include the further section in the training loop

optimizer.zero_grad() # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # does the weight update